In [4]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install gmaps

     |████████████████████████████████| 1.1 MB 4.2 MB/s eta 0:00:01


  Created wheel for gmaps: filename=gmaps-0.9.0-py2.py3-none-any.whl size=2076111 sha256=22e0a99433978901150f9e470a215cb3aa28cfb7235575bc070d597bc34a2978
  Stored in directory: /Users/akarim/Library/Caches/pip/wheels/66/ab/29/3032938612273665f0fa4d2fab4e3ecfe86480eaaa40aaee91
Successfully built gmaps


In [14]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [15]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Great Falls,US,2022-08-29 16:43:44,47.5002,-111.3008,72.41,35,0,11.50
1,1,Codrington,AU,2022-08-29 16:43:45,-38.2667,141.9667,47.75,98,100,10.58
2,2,Sept-Iles,CA,2022-08-29 16:43:45,50.2001,-66.3821,73.40,72,75,10.36
3,3,Bluff,NZ,2022-08-29 16:43:45,-46.6000,168.3333,44.80,85,56,4.76
4,4,Port Alfred,ZA,2022-08-29 16:43:46,-33.5906,26.8910,53.19,86,100,4.00


In [16]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [17]:
gmaps.configure(api_key=g_key)

In [18]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [19]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 70


In [23]:
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
4,4,Port Alfred,ZA,2022-08-29 16:43:46,-33.5906,26.8910,53.19,86,100,4.00
6,6,Busselton,AU,2022-08-29 16:43:46,-33.6500,115.3333,50.29,81,29,4.14
8,8,Luderitz,NaN,2022-08-29 16:43:47,-26.6481,15.1594,63.28,46,0,22.30
18,18,Charters Towers,AU,2022-08-29 16:43:50,-20.1000,146.2667,63.61,95,43,9.15
19,19,Mar Del Plata,AR,2022-08-29 16:39:28,-38.0023,-57.5575,59.95,34,0,17.27


In [24]:
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       3
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [25]:
preferred_cities_df.count()

City_ID       169
City          169
Country       166
Date          169
Lat           169
Lng           169
Max Temp      169
Humidity      169
Cloudiness    169
Wind Speed    169
dtype: int64

In [26]:
preferred_cities_df = preferred_cities_df.dropna()

In [27]:
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
4,4,Port Alfred,ZA,2022-08-29 16:43:46,-33.5906,26.8910,53.19,86,100,4.00
6,6,Busselton,AU,2022-08-29 16:43:46,-33.6500,115.3333,50.29,81,29,4.14
18,18,Charters Towers,AU,2022-08-29 16:43:50,-20.1000,146.2667,63.61,95,43,9.15
19,19,Mar Del Plata,AR,2022-08-29 16:39:28,-38.0023,-57.5575,59.95,34,0,17.27
21,21,Thurso,GB,2022-08-29 16:43:50,58.5927,-3.5259,55.65,73,83,10.60


In [28]:
hotel_df = preferred_cities_df[["City","Country","Max Temp","Lat","Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Port Alfred,ZA,53.19,-33.5906,26.8910,
6,Busselton,AU,50.29,-33.6500,115.3333,
18,Charters Towers,AU,63.61,-20.1000,146.2667,
19,Mar Del Plata,AR,59.95,-38.0023,-57.5575,
21,Thurso,GB,55.65,58.5927,-3.5259,
23,Vestmannaeyjar,IS,54.59,63.4427,-20.2734,
26,Arman,RU,52.02,59.7000,150.1667,
27,Hasaki,JP,68.40,35.7333,140.8333,
31,Victor Harbor,AU,53.01,-35.5667,138.6167,
33,Avarua,CK,66.25,-21.2078,-159.7750,


In [29]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [32]:
#Iterate through the df

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    
    #Add the latitude and longitude to location key for param dict
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found.... skipping.")

Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.


In [41]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Port Alfred,ZA,53.19,-33.5906,26.8910,The Halyards Hotel
6,Busselton,AU,50.29,-33.6500,115.3333,Observatory Guest House
18,Charters Towers,AU,63.61,-20.1000,146.2667,Cattleman's Rest Motor Inn
19,Mar Del Plata,AR,59.95,-38.0023,-57.5575,Gran Hotel Mar del Plata
21,Thurso,GB,55.65,58.5927,-3.5259,Weigh Inn Hotel
23,Vestmannaeyjar,IS,54.59,63.4427,-20.2734,Glamping & Camping
26,Arman,RU,52.02,59.7000,150.1667,
27,Hasaki,JP,68.40,35.7333,140.8333,Inubosaki Kanko Hotel
31,Victor Harbor,AU,53.01,-35.5667,138.6167,McCracken Country Club
33,Avarua,CK,66.25,-21.2078,-159.7750,Paradise Inn


In [37]:
locations = hotel_df[["Lat" , "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating = False, max_intensity = 300, point_radius = 4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [38]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [39]:
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [40]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))